# Project 8

## Assignment Scenario 

You are a data scientist in a business shipping company. You are asked to generate images of handwritten digits using a Deep Convolutional Generative Adversarial Network (DCGAN).

You are asked to develop a DCGAN model based on the sample codesLinks to an external site.  using the attached Jupyter Notebook and writing a script in Python, and running all the cells. You only need to submit a JupyterNotebook.

Data Source: MNIST DatasetLinks to an external site. 

Sample Codes: Deep Convolutional Generative Adversarial NetworkLinks to an external site.

**Steps and Questions:**

1. Load and prepare the MNIST dataset.
2. Create the Generator.
3. Create the discriminator.
4. Define the loss and optimizers.
5. Define the training loop.
6. Train the model.
7. Create a GIF.

## Load Neccesary Libraries, Set Seed

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import glob
import imageio  # For creating GIFs

# Fix the random seeds for reproducibility
SEED = 101
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load Dataset

In [2]:
# Load the MNIST dataset
(latent_dim, batch_size) = (128, 32)
(x_train, _), (_, _) = keras.datasets.mnist.load_data()

# Normalize the data to [0, 1] and reshape it
x_train = x_train.astype("float32") / 255.0
x_train = np.expand_dims(x_train, axis=-1)  # Shape: (N, 28, 28, 1)

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(x_train)
dataset = dataset.shuffle(1024).batch(batch_size)

## Step 2: Create the Generator

In [3]:
def build_generator():
    model = keras.Sequential(
        [
            keras.Input(shape=(latent_dim,)),
            layers.Dense(7 * 7 * 128),
            layers.Reshape((7, 7, 128)),
            layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"),
            layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"),
            layers.Conv2DTranspose(1, kernel_size=4, padding="same", activation="sigmoid"),
        ],
        name="generator",
    )
    return model

## Step 3: Create the Discriminator

In [4]:
def build_discriminator():
    model = keras.Sequential(
        [
            keras.Input(shape=(28, 28, 1)),
            layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Flatten(),
            layers.Dropout(0.2),
            layers.Dense(1, activation="sigmoid"),
        ],
        name="discriminator",
    )
    return model

In [5]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Generate fake images
        generated_images = self.generator(random_latent_vectors)
        combined_images = tf.concat([generated_images, real_images], axis=0)
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Add random noise to the labels
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        # Train the generator
        misleading_labels = tf.zeros((batch_size, 1))
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_latent_vectors)
            predictions = self.discriminator(fake_images)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        return {"d_loss": d_loss, "g_loss": g_loss}

In [6]:
# Initialize the model
generator = build_generator()
discriminator = build_discriminator()
gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

# Compile the model
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

# Custom callback to save generated images
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images = generated_images.numpy().astype("uint8")
        for i in range(self.num_img):
            img = Image.fromarray(generated_images[i, :, :, 0], "L")
            img.save(f"generated_image_{epoch:03d}_{i}.png")

# Train the GAN
epochs = 50
gan.fit(dataset, epochs=epochs, callbacks=[GANMonitor()])

Epoch 1/50
1875/1875 [==============================] - 251s 132ms/step - d_loss: 0.5354 - g_loss: 1.1585
Epoch 2/50
1875/1875 [==============================] - 237s 127ms/step - d_loss: 0.5647 - g_loss: 1.1029
Epoch 3/50
1875/1875 [==============================] - 248s 132ms/step - d_loss: 0.5942 - g_loss: 1.0034
Epoch 4/50
1875/1875 [==============================] - 250s 133ms/step - d_loss: 0.6131 - g_loss: 0.9715
Epoch 5/50
1875/1875 [==============================] - 253s 135ms/step - d_loss: 0.6182 - g_loss: 0.9426
Epoch 6/50
1875/1875 [==============================] - 253s 135ms/step - d_loss: 0.6186 - g_loss: 0.9498
Epoch 7/50
1875/1875 [==============================] - 252s 135ms/step - d_loss: 0.6190 - g_loss: 0.9416
Epoch 8/50
1875/1875 [==============================] - 254s 135ms/step - d_loss: 0.6171 - g_loss: 0.9430
Epoch 9/50
1875/1875 [==============================] - 253s 135ms/step - d_loss: 0.6149 - g_loss: 0.9633
Epoch 10/50
1875/1875 [=======================

In [7]:
# Gather the saved images
generated_files = sorted(glob.glob("generated_image_*.png"))
images = [imageio.imread(file) for file in generated_files]

# Create and save the GIF
gif_path = "generated_images.gif"
imageio.mimsave(gif_path, images, duration=0.5)  # Adjust duration as needed

# Display the GIF
from IPython.display import Image as IPImage, display
display(IPImage(gif_path))
